<h3>Multi-scale Topographic Surface Roughness Analysis of the North Carolina Coastal Plain using Vector Dispersion</h3>

The world's coastal plains are the present day subaerial expressions of the continental margins. With the rise and fall of eustatic and regional sea level, there is a back and forth exchange between the coastal plains and their submerged counterparts, the continental shelves, each one gaining or losing landscape/seascape based on these sea level shifts and the concomitant changes in the ocean shoreline position.

There are a number of ways to approach the problem of geomorphically and geographically mapping the subaerial continental coastal plain, but perhaps the most likely to meet with success is some measure of surface texture or roughness. Roughness can be thought of, from a geostatistical perspective as a measure of variance. Areas presenting more topographic expression (e.g., hillslopes, valley and ridge) display more variation in elevation and in turn, dericative slopes and curvatures. Areas where topographic expression is more subdued, such as the coastal plains that are the subject of the present research, display less over variance n the landscape. I hope to be able to exploit thhis difference in some meaningful and quantitiative way to first geomorphologically identify and then map the continental coastal plains (those present-day subaerial extensions of the continental margins) across the North American Continent. 

To do this then, I need to know how to quantify and then measure topographic variance or roughness. How do I do this?

Grohmann, Smith, and Riccomini, (2011) identify six separate measures (there are others) of topographic surface roughness: 1.) area ratio, 2.) vector dispersion, 3.) standard deviation of residual topography, 4.) standard deviation of elevation, 5.) standard deviation of slope, and 6.) standard deviation of profile curvature. The authors tested the efficacy of the sextet in describing the textural landscape for a region in Scotland. In this notebook, I will apply the vector dispersion method toward assessing the potential to offer a unique roughness signature for identifying and mapping the world's coasts (coastal plains). 

*In this notebook I compute the roughness values for a section of the North Carolina coastal plain using the Vector Dispersion Method as outlined by Grohmann, Smith, and Riccomini. The geography selected is the county of Halifax, North Carolina. Halifax County is ideally (I hope) positioned such that it is intersected by the 'Fall-line' that marks the boundary between coastal plain and piedmont regions.  The strength (or weakness) of the contrast seen between the two regions and the potential of the vector dispersion method to reveal a unique signature for identifying the coastal plain, and mapping it will be assessed.*

**What is surface roughness?**

- a a geographic scale-dependent measure of the spatial variability (variance) of a topographic surface 
- landforms at any scale can represent a roughness element
- other names for surface roughness: ruggedness, rugosity, microrelief, microtopography

**The data**:<br><br>
Data used here--Bare-Earth LiDAR data captured in 2003--is supplied by the North Carolina OneMap website at: https://rmp.nc.gov/sdd/

The region selected is Halifax County, North Carolina. Halifax straddles the "Fall-line", that boundary where coastal plain and peidmont meet. The county is large enough that the 2003 NC LiDAR requires 245 10k x 10k foot map panels.  

![NC Locator](misc/NC_Locator.png)

![NC Locator II](misc/NC_Locator_II.png)

Maps showing location of study area and approximate position of the geologic "fall-line". Fall-line data provided by the U.S. Nuclear Regulatory Commission (don't ask!)


Data was also obtained from the Halifax County Tax Administration Department (Geographic Information Systems): http://www.halifaxnc.com/index.php?option=com_content&view=article&id=48&Itemid=48 for the county land parcels and street centerlines. The latter were not used in this analysis. The land parcels were dissolved into a single county boundary polygon layer (dissolved on feature attribute STATE=NC) in QGIS. This county boundary will be used as an interpolation mask for crafting the final Halifax County DEM.

The format is (per the metadata docs) ASCII XYZ triads in NCSPCS feet<br>
<br>
Bounding_Coordinates:<br>
    West_Bounding_Coordinate: -80.518272<br>
    East_Bounding_Coordinate: -76.664744<br>
    North_Bounding_Coordinate: +36.553775<br>
    South_Bounding_Coordinate: +35.647984<br>
<br>
SPCS_Zone_Identifier: 3200  <br>
    Lambert_Conformal_Conic:<br>
    Standard_Parallel: 34.333333<br>
    Standard_Parallel: 36.166667<br>
    Longitude_of_Central_Meridian: -079.000000<br>
    Latitude_of_Projection_Origin: +33.750000<br>
    False_Easting: 2000000.002617<br>
    False_Northing: 0.000000<br>

Horizontal Datum: NAD83<br>
Vertical Datum: NAVD88<br>
Vertical Accuracy: <= 25cm RMSE<br>
Planar and Altitude units: U.S. Survey Feet  (abv. MSL)<br>

The Halifax County data was preprocessed to convert feet to meters and to concatenate the original 245 files into a single 2.25GB comma-separated values ASCII file using Python script: preprocess_nclidar.py (https://github.com/). Next, the combined data was imported into GRASS using v.in.ascii, and interpolated using v.surf.rst at a working resolution of 100 meters.

A North Carolina county boundary data set (shape file) was downloaded from NCDOT: https://connect.ncdot.gov/resources/gis/pages/gis-data-layers.aspx

The geologic provinces map (States CEUS_physio_USGS_R0.shp) from which the fall-line boundary was derived was graciously provided by: http://www.ceus-ssc.com/Report/GIS.html


GRASS Procs:

v.in.ascii --overwrite input=/Volumes/Beaker/projects/coastal_plains/LiDAR/HalifaxNC.csv output=halifax_elev_xyz separator=comma z=3

g.region vector=halifax_elev_xyz@user res=100.0 save=Halifax_County_100m

v.surf.rst --overwrite input=halifax_elev_xyz@user elevation=Halifax_Elev_100m npmin=120

r.slope.aspect --overwrite elevation=Halifax_Elev_100m@user slope=Halifax_Slope_100m aspect=Halifax_Aspect_100m pcurvature=Halifax_PCurv_100m

**The Vector Dispersion Metric:**

**vector dispersion (Vk)**: a measure the variability in the direction of a set of vectors set orthogonal to the studied surface. Flat areas would present many parallel vectors with low variation (dispersion). As an area becomes increasingly rough directional variability (dispersion) increases in accord.

$$x_i = \sin \theta_i \cos \phi_i \qquad y_i = \sin \theta_i \sin \phi_i  \qquad   z_i = \cos \theta_i$$

$$\overline{x} = \sum_{i=1}^N x_i \qquad \overline{y}=\sum_{i=1}^N y_i \qquad \overline{z} =\sum_{i=1}^N z_i$$

$$ R = \sqrt{\overline{x}^2 + \overline{y}^2 + \overline{z}^2}$$

$$ k = (N-1) / (N-R) $$

where: $\theta$ is the compass-adjusted aspect, $\phi$ is the co-latitude, R = vector strength and Vk = 1/k = vector
dispersion;

compass-adjusted aspect: $\theta = (450 - aspect) \%360$

co-latitude: $\phi = 90 - slope$


In GRASS:

For $\theta$: r.mapcalc --o "Halifax_adjAspect_100m = (450.0 - Halifax_Aspect_100m ) % 360.0"

For $\phi$: r.mapcalc --o "Halifax_coLat_100m = 90.0 - Halifax_Slope_100m"

For directional cosine $x_i$: r.mapcalc --o "Halifax_xi_100m = sin(Halifax_adjAspect_100m)*cos(Halifax_coLat_100m)"

For directional cosine $y_i$: r.mapcalc --o "Halifax_yi_100m = sin(Halifax_adjAspect_100m)*sin(Halifax_coLat_100m)"

For directional cosine $z_i$: r.mapcalc --o "Halifax_zi_100m = cos(Halifax_adjAspect_100m)"

For direction cosine sums:

r.neighbors --overwrite input=Halifax_xi_100m@user output=Halifax_xiSum_100m method=sum

r.neighbors --overwrite input=Halifax_yi_100m@user output=Halifax_yiSum_100m method=sum

r.neighbors --overwrite input=Halifax_zi_100m@user output=Halifax_ziSum_100m method=sum

For N: (the number of cells in the Halifax DEM neighborhood): 3x3 neighborhood window    N=9

Vector Strength (R):  r.mapcalc --o "Halifax_R_100m = ( Halifax_xiSum_100m^2 + Halifax_yiSum_100m^2 + Halifax_ziSum_100m^2 )^0.5"

Dispersion (k): r.mapcalc --o "Halifax_k_100m = 1/(8 / (8 - Halifax_R_100m))"

r.univar map=Halifax_k_100m:  min=-0.124668   max=0.99726

Dispersion (normalized) (knorm): r.mapcalc --o "Halifax_knorm_100m = (Halifax_k_100m - -0.124668) / (0.99726 - -0.124668)"

<h4>Results:</h4>

![Vector Dispersion](misc/Halifax_VDispersion.png)

![Vector Dispersion Profile](misc/Halifax_VKnorm.png)

Vector Dispersion Map and Profile: The smaller the [normalized] dispersion value the smoother the surface (Red areas are smooth(er), blue areas are rough(er)). Based on the dispersion as dipicted on the map and surface profile (running approx NW to SE), if we are assuming that the landscape of the coastal plain is smoother than that of its adjacent piedmont neighbor, then either: 1.) vector dispersion is much too sensitive to micro-scale topography, or 2.) our assumption is incorrect,  or 3.) I made a mistake somewhere in the computation stream...

I suspect that indeed vector dispersion measures are sensitive, even at a resolution of 100 meters, but perhaps more importantly, the assumption that there is a marked difference in the topographic expression (variance) between piedmont/uplands and the coastal plain is simply not true. I continue to hold out that there are differences, though those differences are more subtle and likely manifest through a more elaborate construct than surface complexity. Vector dispersion is simply not the right tool for bringing this to light.